# Семантическое ядро с интеграцией сервера OpenBnB MCP

Этот блокнот демонстрирует, как использовать семантическое ядро с реальным сервером OpenBnB MCP для поиска настоящих вариантов размещения Airbnb с помощью MCPStdioPlugin. Для доступа к LLM используется Azure AI Foundry. Чтобы настроить переменные окружения, вы можете следовать [уроку по настройке](/00-course-setup/README.md).


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Создание подключения плагина MCP

Мы подключимся к [серверу OpenBnB MCP](https://github.com/openbnb-org/mcp-server-airbnb) с использованием MCPStdioPlugin. Этот сервер предоставляет функциональность поиска Airbnb через пакет @openbnb/mcp-server-airbnb.


## Создание клиента

В этом примере мы будем использовать Azure AI Foundry для доступа к LLM. Убедитесь, что переменные окружения настроены правильно.


## Настройка окружения

Настройте параметры Azure OpenAI. Убедитесь, что у вас установлены следующие переменные окружения:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Понимание интеграции OpenBnB MCP

Этот ноутбук подключается к **реальному серверу OpenBnB MCP**, который предоставляет функционал поиска Airbnb.

### Как это работает:

1. **MCPStdioPlugin**: Использует стандартное ввод/вывод для общения с сервером MCP
2. **Реальный NPM пакет**: Загружает и запускает `@openbnb/mcp-server-airbnb` через npx
3. **Живые данные**: Возвращает актуальные данные о недвижимости Airbnb из их API
4. **Обнаружение функций**: Агент автоматически обнаруживает доступные функции на сервере MCP

### Доступные функции:

Сервер OpenBnB MCP обычно предоставляет:
- **search_listings** - Поиск недвижимости Airbnb по местоположению и критериям
- **get_listing_details** - Получение подробной информации о конкретной недвижимости
- **check_availability** - Проверка доступности на определенные даты
- **get_reviews** - Получение отзывов о недвижимости
- **get_host_info** - Получение информации о хозяевах недвижимости

### Предварительные требования:

- Установленный **Node.js** на вашем компьютере
- **Интернет-соединение** для загрузки пакета сервера MCP
- **NPX** (идет в комплекте с Node.js)

### Тестирование подключения:

Вы можете протестировать сервер MCP вручную, запустив:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Это загрузит и запустит сервер OpenBnB MCP, к которому затем подключается Semantic Kernel для получения реальных данных Airbnb.


## Запуск агента с сервером OpenBnB MCP

Теперь мы запустим AI-агента, который подключается к серверу OpenBnB MCP, чтобы искать реальные варианты жилья на Airbnb в Стокгольме для 2 взрослых и 1 ребенка. Вы можете изменить список `user_inputs`, чтобы настроить критерии поиска.


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

Резюме  
Поздравляем! Вы успешно создали AI-агента, который интегрируется с поиском реального жилья с использованием протокола Model Context Protocol (MCP):

Используемые технологии:  
Semantic Kernel - для создания интеллектуальных агентов с Azure OpenAI  
Azure AI Foundry - для возможностей LLM и завершения чатов  
MCP (Model Context Protocol) - для стандартизированной интеграции инструментов  
OpenBnB MCP Server - для реального поиска жилья на Airbnb  
Node.js/NPX - для запуска внешнего MCP-сервера  

Чему вы научились:  
Интеграция MCP: подключение агентов Semantic Kernel к внешним MCP-серверам  
Доступ к данным в реальном времени: поиск реальных объектов Airbnb через живые API  
Протокольная коммуникация: использование stdio для связи между агентом и MCP-сервером  
Обнаружение функций: автоматическое обнаружение доступных функций на MCP-серверах  
Потоковые ответы: захват и логирование вызовов функций в реальном времени  
Рендеринг HTML: форматирование ответов агента с помощью стилизованных таблиц и интерактивных элементов  

Следующие шаги:  
Интеграция дополнительных MCP-серверов (погода, рейсы, рестораны)  
Создание системы с несколькими агентами, объединяющей MCP и A2A протоколы  
Разработка собственных MCP-серверов для ваших источников данных  
Реализация постоянной памяти разговоров между сессиями  
Развертывание агента в Azure Functions с оркестрацией MCP-сервера  
Добавление аутентификации пользователей и возможностей бронирования  

Ключевые преимущества архитектуры MCP:  
Стандартизация: универсальный протокол для подключения AI-агентов к внешним инструментам  
Данные в реальном времени: доступ к актуальной информации от различных сервисов  
Расширяемость: простая интеграция новых источников данных и инструментов  
Взаимодействие: работает с различными AI-фреймворками и платформами агентов  
Разделение задач: четкое разграничение между логикой AI и доступом к внешним данным  



---

**Отказ от ответственности**:  
Этот документ был переведен с использованием сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Хотя мы стремимся к точности, пожалуйста, имейте в виду, что автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его исходном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникшие в результате использования данного перевода.
